In [59]:
import os,datetime
import cv2
import xml.etree.ElementTree as ET
from PIL import Image
import numpy as np
import random
from pathlib import Path

time_str = (datetime.datetime.now()).strftime("%Y%m%d")
time_str = '20190507'
#xml_dir = '/data/darknet/python/' + time_str + '_xml/'
#image_dir = '/data/darknet/python/' + time_str + '_image/'
xml_dir = '/data/darknet/python/' + time_str + '_xml_expand_obj/'
image_dir = '/data/darknet/python/' + time_str + '_image_expand_obj/'

generated_image_dir = '/data/darknet/python/' + time_str + '_image_croped/'
generated_xml_dir = '/data/darknet/python/' + time_str + '_xml_croped/'

#os.system('mkdir -p ' + xml_dir)
#os.system('mkdir -p ' + image_dir)
os.system('mkdir -p ' + generated_image_dir)
os.system('mkdir -p ' + generated_xml_dir)

SEED_XML_DIR = xml_dir
SEED_IMG_DIR = image_dir
GENE_IMG_DIR = generated_image_dir
GENE_XML_DIR = generated_xml_dir


# RT:RightTop 
# LB:LeftBottom 
# bbox: [xmin, xax, ymin, ymax]
def IOU(bbox_a, bbox_b):
    '''
    W = min(A.RT.x, B.RT.x) - max(A.LB.x, B.LB.x) 
    H = min(A.RT.y, B.RT.y) - max(A.LB.y, B.LB.y) 
    if W <= 0 or H <= 0: 
        return 0 
    SA = (A.RT.x - A.LB.x) * (A.RT.y - A.LB.y) 
    SB = (B.RT.x - B.LB.x) * (B.RT.y - B.LB.y) 
    cross = W * H return cross/(SA + SB - cross)
    '''
    W = min(bbox_a[1], bbox_b[1]) - max(bbox_a[0], bbox_b[0]) 
    H = min(bbox_a[3], bbox_b[3]) - max(bbox_a[2], bbox_b[2]) 
    if W <= 0 or H <= 0: 
        return 0
    SA = (bbox_a[1] - bbox_a[0]) * (bbox_a[3] - bbox_a[2]) 
    SB = (bbox_b[1] - bbox_b[0]) * (bbox_b[3] - bbox_b[2])  
    cross = W * H 
    return cross/(SA + SB - cross)
    
def get_intercouse_box(bbox_a, bbox_b):
    xmin_ = max(bbox_a[0], bbox_b[0])
    xmax_ = min(bbox_a[1], bbox_b[1])
    ymin_ = max(bbox_a[2], bbox_b[2])
    ymax_ = min(bbox_a[3], bbox_b[3])
    if((xmax_ - xmin_ < 20) or (ymax_ - ymin_ < 20)):
        return False
    return [xmin_, xmax_, ymin_, ymax_]

def get_bbox_from_xmlobj(obj_element):
    xmlbox = obj_element.find('bndbox')
    return [float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text), 
            float(xmlbox.find('ymin').text), float(xmlbox.find('ymax').text)]

def get_obj_from_xml(xml):
    in_file = open(xml_name)
    tree=ET.parse(in_file)
    root = tree.getroot()
    return [obj for obj in root.iter('object')]

def get_obj_from_image_file(file, bbox):
    img = cv2.imread(file)
    img_obj = img[int(bbox[2]):int(bbox[3]), int(bbox[0]):int(bbox[1])]
    return img_obj

def get_bboxes_from_etree(etree):
    root = etree.getroot()  
    objects = root.findall('object')
    bboxes = []
    for obj in objects:
        xmlbox = obj.find('bndbox')
        b = [float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text), float(xmlbox.find('ymin').text), float(xmlbox.find('ymax').text)]
        bboxes.append(b)
    return bboxes
def past_to_background_from_image_file(file, bboxes, background_img_array, extend_spaces=0):    
    img = cv2.imread(file)
    if(img.shape != background_img_array.shape):
        print('shape not match')
        return
    img_objs = []
    for bbox in bboxes:
        img_obj = img[int(bbox[2]):int(bbox[3]), int(bbox[0]):int(bbox[1])]
        img_objs.append(img_obj)
    i = 0
    for bbox in bboxes:
        background_img_array[int(bbox[2]):int(bbox[3]), int(bbox[0]):int(bbox[1])] = img_objs[i]
        i = i+1
    cv2.imwrite(GENE_IMG_DIR+file.split('/')[-1], background_img_array)
    return 
def generate_new_xmlobj(xmlobj_old, new_position, new_size):
    element_object = ET.Element('object')
    tag_name = ET.SubElement(element_object, 'name')
    tag_name.text = xmlobj_old.find('name').text

    tag_difficult = ET.SubElement(element_object, 'difficult')
    tag_difficult.text = xmlobj_old.find('difficult').text

    element_bndbox = ET.SubElement(element_object, 'bndbox')
    tag_xmin = ET.SubElement(element_bndbox, 'xmin')
    tag_ymin = ET.SubElement(element_bndbox, 'ymin')
    tag_xmax = ET.SubElement(element_bndbox, 'xmax')
    tag_ymax = ET.SubElement(element_bndbox, 'ymax')
    tag_xmin.text = str(new_position[0])
    tag_ymin.text = str(new_position[1])
    tag_xmax.text = str(new_position[0] + new_size[1])
    tag_ymax.text = str(new_position[1] + new_size[0])
    return element_object

def generate_etree(etree_old):
    root_old = etree_old.getroot()    
    root = ET.Element('annotation')
    root.append(root_old.find('folder'))
    root.append(root_old.find('filename'))
    root.append(root_old.find('source'))
    root.append(root_old.find('size'))
    root.append(root_old.find('segmented'))    
    tree = ET.ElementTree(root)    
    return tree
    
def past_and_insert(img_obj, img_array, new_position, obj_element, etree):
    new_xmlobj = generate_new_xmlobj(obj_element, new_position, img_obj.shape[:2])
    new_xml_etree = insert_to_xml(new_xmlobj, etree)
    new_pil_img = past_obj_to_background(img_obj, img_array, new_position)
    return new_pil_img, new_xml_etree
    

def insert_to_xml(xml_obj, xml_etree):
    root = xml_etree.getroot()
    root.append(xml_obj)
    return xml_etree
    
def past_obj_to_background(img_obj, img_array, position = (200, 200)):
    img = Image.fromarray(img_array)
    img_obj = Image.fromarray(img_obj)
    img.paste(img_obj, position)
    return img
    
def get_cls_from_xmlobj(obj_element):    
    return obj_element.find('name').text

def generate_new_position(img_size, img_obj_size):
    array_x = np.arange(int(img_size[1] - img_obj_size[1]))
    array_y = np.arange(int(img_size[0] - img_obj_size[0]))
    random_x = random.sample(list(array_x), 1)[0]
    random_y = random.sample(list(array_y), 1)[0]
    new_position = (random_x, random_y)
    #print(new_position)
    return new_position

def generate_new_bbox(img_size, img_obj_size):
    array_x = np.arange(int(img_size[1] - img_obj_size[1]))
    array_y = np.arange(int(img_size[0] - img_obj_size[0]))
    random_x = random.sample(list(array_x), 1)[0]
    random_y = random.sample(list(array_y), 1)[0]
    new_position = (random_x, random_y)
    new_bbox = [random_x, random_x + img_obj_size[1], random_y, random_y + img_obj_size[0]]
    #print(new_bbox)
    return new_bbox
def generate_union_bbox(matrix1, matrix2): #xin,xmax,ymin,ymax
    img_obj_shape = (int(max(matrix1[3], matrix2[3])-min(matrix1[2], matrix2[2])), 
                int(max(matrix1[1], matrix2[1])-min(matrix1[0], matrix2[0])))
    return img_obj_shape

def generate_union_bbox_shape(matrix_list): #xin,xmax,ymin,ymax
    matrix_array = np.array(matrix_list)
    img_obj_shape = (int(max(matrix_array[:,3]) - min(matrix_array[:,2])),
                     int(max(matrix_array[:,1]) - min(matrix_array[:,0])))
    return img_obj_shape

def get_union_bbox_left_top(matrix_list): #xin,xmax,ymin,ymax
    matrix_array = np.array(matrix_list)
    left_top = (int(min(matrix_array[:,0])),
                     int(min(matrix_array[:,2])))
    return left_top
def generate_union_bbox_(matrix_list):
    img_obj_shape = generate_union_bbox_shape(matrix_list)
    left_top = get_union_bbox_left_top(matrix_list)
    union_bbox = [int(left_top[0]), 
                  int(left_top[0]+int(img_obj_shape[1])),
                  int(left_top[1]),
                  int(left_top[1]+int(img_obj_shape[0]))]
    return union_bbox

def generate_new_bbox_within_distance(img_size, img_obj_size, old_bbox, scale=2.5):
    array_x = np.arange(int(img_size[1] - img_obj_size[1]))
    #array_y = np.arange(int(img_size[0] - img_obj_size[0])/3, int(img_size[0] - img_obj_size[0])/3*2)
    array_y = np.arange(max(0, int((old_bbox[3]+old_bbox[2])/2 - scale*img_obj_size[0])),
                        min(int((old_bbox[3]+old_bbox[2])/2 + scale*img_obj_size[0]), int(img_size[0] - img_obj_size[0])))
    random_x = random.sample(list(array_x), 1)[0]
    random_y = random.sample(list(array_y), 1)[0]
    new_position = (random_x, random_y)
    new_bbox = [random_x, random_x + img_obj_size[1], random_y, random_y + img_obj_size[0]]
    #print(new_bbox)
    return new_bbox

def check_before_insert(img_size, img_obj_size, etree, union_bbox):
    new_bbox = generate_new_bbox_within_distance(img_size, img_obj_size, union_bbox)
    bboxes = get_bboxes_from_etree(etree)
    retry_times = 0
    while(not check_bbox(new_bbox, bboxes)):
        print('new_bbox not suitable, retry...')
        retry_times = retry_times + 1
        if(retry_times > 100):
            return (False, False)
        new_bbox = generate_new_bbox_within_distance(img_size, img_obj_size, union_bbox)
    return (new_bbox[0], new_bbox[2])
    
def inset_obj_to_an_image_and_xml(img_obj, img, obj_element, etree):
    new_bbox = generate_new_bbox(img.shape[:2], img_obj.shape[:2])
    bboxes = get_bboxes_from_etree(etree)
    retry_times = 0
    while(not check_bbox(new_bbox, bboxes)):
        print('new_bbox not suitable, retry...')
        retry_times = retry_times + 1
        if(retry_times > 50):
            return False, False
        new_bbox = generate_new_bbox(img.shape[:2], img_obj.shape[:2])
    #print('new_bbox succussful')
    new_pil_img, new_xml_etree = past_and_insert(img_obj, img, (new_bbox[0], new_bbox[2]), obj_element, etree)
    
    return new_pil_img, new_xml_etree

def do_inset_obj_to_an_image_and_xml(img_obj, img, obj_element, etree, insert_point):
    new_pil_img, new_xml_etree = past_and_insert(img_obj, img, insert_point, obj_element, etree)    
    return new_pil_img, new_xml_etree

def inset_dobule_obj_to_an_image_and_xml(img_obj1, img_obj2, img, obj_element1, obj_element2, etree):
    new_bbox = generate_new_bbox(img.shape[:2], img_obj.shape[:2])
    bboxes = get_bboxes_from_etree(etree)
    retry_times = 0
    while(not check_bbox(new_bbox, bboxes)):
        print('new_bbox not suitable, retry...')
        retry_times = retry_times + 1
        if(retry_times > 50):
            return False, False
        new_bbox = generate_new_bbox(img.shape[:2], img_obj.shape[:2])
    #print('new_bbox succussful')
    new_pil_img, new_xml_etree = past_and_insert(img_obj, img, (new_bbox[0], new_bbox[2]), obj_element, etree)
    
    return new_pil_img, new_xml_etree

def check_bbox(new_bbox, bboxes):
    for bbox in bboxes:
        if(IOU(new_bbox, bbox) > 0):
            return False
    return True
def caculate_move_vector(bbox, new_point):
    original_xmin, original_ymin = bbox[0], bbox[2]
    diffx, diffy = new_point[0] - original_xmin, new_point[1] - original_ymin
    return (diffx, diffy)

def rotate_img(img, thealta):
    (h_, w_) = img.shape[:2]
    point_list = [(0, 0), (0, h_), (w_, h_), (w_, 0)]
    center = (w_ // 2, h_ // 2)
    roted_point_list = [((point[0]-center[0])*np.cos(np.pi*thealta/180) 
                         - (point[1]-center[1])*np.sin(np.pi*thealta/180) 
                         + center[0], (point[0]-center[0])*np.sin(np.pi*thealta/180) 
                         + (point[1]-center[1])*np.cos(np.pi*thealta/180) + center[1]) for point in point_list]
    #print(roted_point_list)

    temp = np.zeros((2, 4))
    temp[0] = [roted_point[0] for roted_point in roted_point_list]
    temp[1] = [roted_point[1] for roted_point in roted_point_list]
    (xmin, xmax, ymin, ymax) = (np.min(temp[0]), np.max(temp[0]), np.min(temp[1]), np.max(temp[1]))
    print('xmin: {0}, xmax: {1}, ymin: {2}, ymax: {3}'.format(xmin, xmax, ymin, ymax))

    roted_h, roted_w = ymax - ymin, xmax - xmin
    print('roted_h: {0}, roted_w: {1}'.format(roted_w, roted_h))
    
    top_bottom, left_right = int((roted_h - h_)/2), int((roted_w - w_)/2)
    print('top_bottom:{0}, left_right_:{1}'.format(top_bottom, left_right))
    padding = lambda arg : max(arg, 0)
    #dst = cv2.copyMakeBorder(img, padding(top_bottom), padding(top_bottom), padding(left_right), 
    #padding(left_right), cv2.BORDER_CONSTANT)
    dst = cv2.copyMakeBorder(img, padding(top_bottom), padding(top_bottom), padding(left_right), 
                             padding(left_right), cv2.BORDER_CONSTANT)
    
    (h, w) = dst.shape[:2]   
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, thealta, 1.0)
    rotated_img = cv2.warpAffine(dst, M, (w, h))
    if(top_bottom < 0):
        top_bottom = abs(top_bottom)
        rotated_img = rotated_img[top_bottom:-top_bottom, :, :]
    if(left_right < 0):
        left_right = abs(left_right)
        rotated_img = rotated_img[:, left_right:-left_right, :]
    return rotated_img

seed_xml_names = os.listdir(SEED_XML_DIR)
seed_xml_names.sort()


In [60]:
#croped_shape = [480, 640]
LOOPTIMES = 2
for L in range(LOOPTIMES):
    for xml_name in seed_xml_names:
        img_name = SEED_IMG_DIR + xml_name[:-3] + 'jpg'
        xml_name = SEED_XML_DIR + xml_name
        print(img_name)
        print(xml_name)

        etree=ET.parse(xml_name)
        img = cv2.imread(img_name)
        croped_shape = [int(img.shape[0]/1.5), int(img.shape[1]/2)]
        objs = get_obj_from_xml(xml_name)
        bboxes = get_bboxes_from_etree(tree)

        ## 0.generate new empty element tree
        new_etree = generate_etree(etree)
        root = new_etree.getroot()
        xml_obj_size = root.find('size')
        xml_obj_size.find('width').text = str(croped_shape[1])
        xml_obj_size.find('height').text = str(croped_shape[0])
        #new_etree.write('/data/test.xml')

        ## 1.generate crop box
        new_bbox = generate_new_bbox(img.shape[:2], croped_shape)
        img_obj = get_obj_from_image_file(img_name, new_bbox)
        (diffx, diffy) = caculate_move_vector(new_bbox, (0, 0))

        ## 2.find the obj that in crop box
        new_objs = []
        for obj_element in objs:
            bbox = get_bbox_from_xmlobj(obj_element)
            inter_bbox = get_intercouse_box(bbox, new_bbox)
            if(inter_bbox == False):
                continue            
            inter_bbox_trans = [int(inter_bbox[0] + diffx), int(inter_bbox[1] + diffx), 
                                int(inter_bbox[2] + diffy), int(inter_bbox[3] + diffy)]
            cls = get_cls_from_xmlobj(obj_element)
            new_xmlobj = generate_new_xmlobj(obj_element, (inter_bbox_trans[0], inter_bbox_trans[2]), 
                                (int(inter_bbox[3] - inter_bbox[2]), int(inter_bbox[1] - inter_bbox[0])))
            new_objs.append(new_xmlobj)
        if(len(new_objs) > 0):
            for i in new_objs:
                root.append(i)
        else:
            continue   

        ## save xml and img
        time_mark = datetime.datetime.now()
        time_str = time_mark.strftime("%Y%m%d%H%M%S%f_")
        cv2.imwrite(GENE_IMG_DIR + time_str + img_name.split('/')[-1], img_obj)    
        new_etree.write(GENE_XML_DIR + time_str + xml_name.split('/')[-1])


/data/darknet/python/20190507_image_expand_obj/20190509110352947152_003010000.0.jpg
/data/darknet/python/20190507_xml_expand_obj/20190509110352947152_003010000.0.xml
/data/darknet/python/20190507_image_expand_obj/20190509110353642625_003010002.0.jpg
/data/darknet/python/20190507_xml_expand_obj/20190509110353642625_003010002.0.xml
/data/darknet/python/20190507_image_expand_obj/20190509110354923687_003010005.0.jpg
/data/darknet/python/20190507_xml_expand_obj/20190509110354923687_003010005.0.xml
/data/darknet/python/20190507_image_expand_obj/20190509110356401725_003010007.0.jpg
/data/darknet/python/20190507_xml_expand_obj/20190509110356401725_003010007.0.xml
/data/darknet/python/20190507_image_expand_obj/20190509110358320827_003010009.0.jpg
/data/darknet/python/20190507_xml_expand_obj/20190509110358320827_003010009.0.xml
/data/darknet/python/20190507_image_expand_obj/20190509110400002120_003010010.0.jpg
/data/darknet/python/20190507_xml_expand_obj/20190509110400002120_003010010.0.xml
/dat

/data/darknet/python/20190507_image_expand_obj/20190509110505987945_003010073.0.jpg
/data/darknet/python/20190507_xml_expand_obj/20190509110505987945_003010073.0.xml
/data/darknet/python/20190507_image_expand_obj/20190509110506847405_003010074.0.jpg
/data/darknet/python/20190507_xml_expand_obj/20190509110506847405_003010074.0.xml
/data/darknet/python/20190507_image_expand_obj/20190509110508120548_003010076.0.jpg
/data/darknet/python/20190507_xml_expand_obj/20190509110508120548_003010076.0.xml
/data/darknet/python/20190507_image_expand_obj/20190509110509381162_003010078.0.jpg
/data/darknet/python/20190507_xml_expand_obj/20190509110509381162_003010078.0.xml
/data/darknet/python/20190507_image_expand_obj/20190509110510746603_003010079.0.jpg
/data/darknet/python/20190507_xml_expand_obj/20190509110510746603_003010079.0.xml
/data/darknet/python/20190507_image_expand_obj/20190509110511647618_003010080.0.jpg
/data/darknet/python/20190507_xml_expand_obj/20190509110511647618_003010080.0.xml
/dat

/data/darknet/python/20190507_image_expand_obj/20190509110630941336_003010153.0.jpg
/data/darknet/python/20190507_xml_expand_obj/20190509110630941336_003010153.0.xml
/data/darknet/python/20190507_image_expand_obj/20190509110631974177_003010154.0.jpg
/data/darknet/python/20190507_xml_expand_obj/20190509110631974177_003010154.0.xml
/data/darknet/python/20190507_image_expand_obj/20190509110632792170_003010155.0.jpg
/data/darknet/python/20190507_xml_expand_obj/20190509110632792170_003010155.0.xml
/data/darknet/python/20190507_image_expand_obj/20190509110633586282_003010156.0.jpg
/data/darknet/python/20190507_xml_expand_obj/20190509110633586282_003010156.0.xml
/data/darknet/python/20190507_image_expand_obj/20190509110634400920_003010158.0.jpg
/data/darknet/python/20190507_xml_expand_obj/20190509110634400920_003010158.0.xml
/data/darknet/python/20190507_image_expand_obj/20190509110635886358_003010159.0.jpg
/data/darknet/python/20190507_xml_expand_obj/20190509110635886358_003010159.0.xml
/dat

/data/darknet/python/20190507_image_expand_obj/20190509110731285201_003010237.0.jpg
/data/darknet/python/20190507_xml_expand_obj/20190509110731285201_003010237.0.xml
/data/darknet/python/20190507_image_expand_obj/20190509110732007677_003010239.0.jpg
/data/darknet/python/20190507_xml_expand_obj/20190509110732007677_003010239.0.xml
/data/darknet/python/20190507_image_expand_obj/20190509110733708496_003010240.0.jpg
/data/darknet/python/20190507_xml_expand_obj/20190509110733708496_003010240.0.xml
/data/darknet/python/20190507_image_expand_obj/20190509110734560159_003010241.0.jpg
/data/darknet/python/20190507_xml_expand_obj/20190509110734560159_003010241.0.xml
/data/darknet/python/20190507_image_expand_obj/20190509110735867504_003010242.0.jpg
/data/darknet/python/20190507_xml_expand_obj/20190509110735867504_003010242.0.xml
/data/darknet/python/20190507_image_expand_obj/20190509110736742148_003010243.0.jpg
/data/darknet/python/20190507_xml_expand_obj/20190509110736742148_003010243.0.xml
/dat

/data/darknet/python/20190507_image_expand_obj/20190509110859026634_003010313.0.jpg
/data/darknet/python/20190507_xml_expand_obj/20190509110859026634_003010313.0.xml
/data/darknet/python/20190507_image_expand_obj/20190509110900215623_003010314.0.jpg
/data/darknet/python/20190507_xml_expand_obj/20190509110900215623_003010314.0.xml
/data/darknet/python/20190507_image_expand_obj/20190509110901088807_003010317.0.jpg
/data/darknet/python/20190507_xml_expand_obj/20190509110901088807_003010317.0.xml
/data/darknet/python/20190507_image_expand_obj/20190509110902750286_003010318.0.jpg
/data/darknet/python/20190507_xml_expand_obj/20190509110902750286_003010318.0.xml
/data/darknet/python/20190507_image_expand_obj/20190509110904600298_003010320.0.jpg
/data/darknet/python/20190507_xml_expand_obj/20190509110904600298_003010320.0.xml
/data/darknet/python/20190507_image_expand_obj/20190509110906182942_003010321.0.jpg
/data/darknet/python/20190507_xml_expand_obj/20190509110906182942_003010321.0.xml
/dat

...

/data/darknet/python/20190507_image_expand_obj/20190509120950681356_003011747.0.jpg
/data/darknet/python/20190507_xml_expand_obj/20190509120950681356_003011747.0.xml
/data/darknet/python/20190507_image_expand_obj/20190509120952220224_003011748.0.jpg
/data/darknet/python/20190507_xml_expand_obj/20190509120952220224_003011748.0.xml
/data/darknet/python/20190507_image_expand_obj/20190509120954049766_003011749.0.jpg
/data/darknet/python/20190507_xml_expand_obj/20190509120954049766_003011749.0.xml
/data/darknet/python/20190507_image_expand_obj/20190509120955734462_003011750.0.jpg
/data/darknet/python/20190507_xml_expand_obj/20190509120955734462_003011750.0.xml
/data/darknet/python/20190507_image_expand_obj/20190509120957034817_003011751.0.jpg
/data/darknet/python/20190507_xml_expand_obj/20190509120957034817_003011751.0.xml
/data/darknet/python/20190507_image_expand_obj/20190509120957835664_003011752.0.jpg
/data/darknet/python/20190507_xml_expand_obj/20190509120957835664_003011752.0.xml
/dat

/data/darknet/python/20190507_image_expand_obj/20190509121101928624_003011871.0.jpg
/data/darknet/python/20190507_xml_expand_obj/20190509121101928624_003011871.0.xml
/data/darknet/python/20190507_image_expand_obj/20190509121103061002_003011872.0.jpg
/data/darknet/python/20190507_xml_expand_obj/20190509121103061002_003011872.0.xml
/data/darknet/python/20190507_image_expand_obj/20190509121103895885_003011873.0.jpg
/data/darknet/python/20190507_xml_expand_obj/20190509121103895885_003011873.0.xml
/data/darknet/python/20190507_image_expand_obj/20190509121105036113_003011874.0.jpg
/data/darknet/python/20190507_xml_expand_obj/20190509121105036113_003011874.0.xml
/data/darknet/python/20190507_image_expand_obj/20190509121106171600_003011875.0.jpg
/data/darknet/python/20190507_xml_expand_obj/20190509121106171600_003011875.0.xml
/data/darknet/python/20190507_image_expand_obj/20190509121107579390_003011877.0.jpg
/data/darknet/python/20190507_xml_expand_obj/20190509121107579390_003011877.0.xml
/dat

/data/darknet/python/20190507_image_expand_obj/20190509121151491425_003011961.0.jpg
/data/darknet/python/20190507_xml_expand_obj/20190509121151491425_003011961.0.xml
/data/darknet/python/20190507_image_expand_obj/20190509121152140508_003011962.0.jpg
/data/darknet/python/20190507_xml_expand_obj/20190509121152140508_003011962.0.xml
/data/darknet/python/20190507_image_expand_obj/20190509121153016153_003011964.0.jpg
/data/darknet/python/20190507_xml_expand_obj/20190509121153016153_003011964.0.xml
/data/darknet/python/20190507_image_expand_obj/20190509121153787243_003011965.0.jpg
/data/darknet/python/20190507_xml_expand_obj/20190509121153787243_003011965.0.xml
/data/darknet/python/20190507_image_expand_obj/20190509121154705284_003011966.0.jpg
/data/darknet/python/20190507_xml_expand_obj/20190509121154705284_003011966.0.xml
/data/darknet/python/20190507_image_expand_obj/20190509121155477363_003011967.0.jpg
/data/darknet/python/20190507_xml_expand_obj/20190509121155477363_003011967.0.xml
/dat

/data/darknet/python/20190507_image_expand_obj/20190509121251700207_003012036.0.jpg
/data/darknet/python/20190507_xml_expand_obj/20190509121251700207_003012036.0.xml
/data/darknet/python/20190507_image_expand_obj/20190509121253379229_003012037.0.jpg
/data/darknet/python/20190507_xml_expand_obj/20190509121253379229_003012037.0.xml
/data/darknet/python/20190507_image_expand_obj/20190509121255092977_003012038.0.jpg
/data/darknet/python/20190507_xml_expand_obj/20190509121255092977_003012038.0.xml
/data/darknet/python/20190507_image_expand_obj/20190509121256022373_003012039.0.jpg
/data/darknet/python/20190507_xml_expand_obj/20190509121256022373_003012039.0.xml
/data/darknet/python/20190507_image_expand_obj/20190509121257269206_003012041.0.jpg
/data/darknet/python/20190507_xml_expand_obj/20190509121257269206_003012041.0.xml
/data/darknet/python/20190507_image_expand_obj/20190509121259233133_003012043.0.jpg
/data/darknet/python/20190507_xml_expand_obj/20190509121259233133_003012043.0.xml
/dat

/data/darknet/python/20190507_image_expand_obj/20190509121350348054_003012127.0.jpg
/data/darknet/python/20190507_xml_expand_obj/20190509121350348054_003012127.0.xml
/data/darknet/python/20190507_image_expand_obj/20190509121351025758_003012128.0.jpg
/data/darknet/python/20190507_xml_expand_obj/20190509121351025758_003012128.0.xml
/data/darknet/python/20190507_image_expand_obj/20190509121351610946_003012130.0.jpg
/data/darknet/python/20190507_xml_expand_obj/20190509121351610946_003012130.0.xml
/data/darknet/python/20190507_image_expand_obj/20190509121352823866_003012131.0.jpg
/data/darknet/python/20190507_xml_expand_obj/20190509121352823866_003012131.0.xml
/data/darknet/python/20190507_image_expand_obj/20190509121354363127_003012132.0.jpg
/data/darknet/python/20190507_xml_expand_obj/20190509121354363127_003012132.0.xml
/data/darknet/python/20190507_image_expand_obj/20190509121355625361_003012133.0.jpg
/data/darknet/python/20190507_xml_expand_obj/20190509121355625361_003012133.0.xml
/dat